# Setup
Install necessary packages

In [17]:
!pip install wandb torch ultralytics scikit-image

In [15]:
!pip uninstall torch torchvision
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu117

^C
Looking in indexes: https://download.pytorch.org/whl/cu117


In [3]:
import os
from pathlib import Path
import shutil

import torch
import pandas as pd
import numpy as np
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, datasets

from ultralytics import YOLO
from ultralytics.yolo.engine.trainer import BaseTrainer

import wandb

# Data Preprocessing
Set up data for training and validation

In [21]:
cwd = Path.cwd()
datasets_path = cwd / "datasets"
prannays_edibles_path = datasets_path / "prannays_edibles"
print(prannays_edibles_path)

c:\Users\Francis Ralph\Desktop\Misc\BuildingBloCS\BBCS-June-Intermediate\datasets\prannays_edibles


In [22]:
class_name_map = {
    '0': 'bread',
    '1': 'dairy',
    '2': 'dessert',
    '3': 'egg',
    '4': 'fried',
    '5': 'meat',
    '6': 'pasta',
    '7': 'rice',
    '8': 'seafood',
    '9': 'soup',
    '10': 'vegetables',
}

In [7]:
prannays_edibles_dataset = datasets.ImageFolder(root=prannays_edibles_path)

train_split_percentage = 0.7
test_split_percentage = 1 - train_split_percentage

train_dataset, test_dataset = random_split(prannays_edibles_dataset, [train_split_percentage, test_split_percentage])

split_dataset_path = cwd / "datasets" / "prannays_edibles_split"
if split_dataset_path.exists() and split_dataset_path.is_dir():
    shutil.rmtree(split_dataset_path) # reset split

split_dataset_path.mkdir(exist_ok=True)

train_dataset_path = split_dataset_path / "train"
train_dataset_path.mkdir(exist_ok=True)

test_dataset_path = split_dataset_path / "test"
test_dataset_path.mkdir(exist_ok=True)

def create_dataset_folder(dataset, dataset_path):
    for i, (image, image_class) in enumerate(dataset):
        class_path = dataset_path / str(image_class)
        if not class_path.exists():
            class_path.mkdir(exist_ok=True)
        image.save(class_path / f"{image_class}_{i}.jpg")

create_dataset_folder(train_dataset, train_dataset_path)
create_dataset_folder(test_dataset, test_dataset_path)

# Model Training

In [23]:
model_path = Path('yolov8n-cls.pt')
if model_path.exists():
    model_path.unlink()
model = YOLO('yolov8n-cls.pt') # load pretrained model

100%|██████████| 5.28M/5.28M [00:00<00:00, 10.2MB/s]


In [9]:
# login to wandb to monitor training metrics
os.environ["WANDB_API_KEY"] = input()
wandb.init(project='BuildingBloCS Prannays Edibles Classifier', settings=wandb.Settings(start_method="spawn"), mode='online')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: francisralph. Use `wandb login --relogin` to force relogin


In [24]:
epochs = 10
batch = 4

model.train(data=str(split_dataset_path), batch=batch, epochs=epochs)

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


# Model Validation

In [12]:
metrics = model.val(data=str(split_dataset_path), device=0)
print("metrics", metrics)

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


# Model Prediction

In [88]:
results = model('images_to_predict/prannays_edibles/steak.jpg')


image 1/1 c:\Users\Francis Ralph\Desktop\Misc\BuildingBloCS\BBCS-June-Intermediate\images_to_predict\prannays_edibles\steak.jpg: 224x224 3 0.13, 6 0.11, 4 0.11, 10 0.11, 0 0.08, 21.0ms
Speed: 7.0ms preprocess, 21.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [89]:
for result in results:
    for i, contender in enumerate(result.probs.top5):
        print(i, class_name_map[str(contender)], f"({result.probs.top5conf[i]:.2f}% confidence)")

0 fried (0.13% confidence)
1 rice (0.11% confidence)
2 meat (0.11% confidence)
3 dessert (0.11% confidence)
4 bread (0.08% confidence)
